In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

data = pd.read_csv('contacts_part', names = ['contact_id', 'client_id', 'employee_id','started_dttm', 'finished_dttm'
                                        , 'business_line', 'route_type', 'initiator_id'], sep = '\t')

print("====== Reading finished =====")

data['started_dttm'] = pd.to_datetime(data['started_dttm'])
data['finished_dttm'] = pd.to_datetime(data['finished_dttm'])
data_emp = data[['employee_id', 'started_dttm', 'finished_dttm']].groupby('employee_id').agg(
    first_date=('started_dttm', np.min),
    last_date=('finished_dttm', np.max)).reset_index()
data_emp['work'] = (data_emp['last_date'] - data_emp['first_date']).dt.days

print("====== Countind work duration finished =====")

data = data[data['employee_id'].isin(data_emp[data_emp['work'] >= 180]['employee_id'].to_list())]
data = data[(data['business_line'].str.upper() == 'CREDIT CARD') & (data['route_type'].str.upper() == 'TELEPHONY')]
data['call'] = (data['finished_dttm'] - data['started_dttm']).dt.seconds
data['employee_id'] = data['employee_id'].astype('string')
data['initiator_id'] = data['initiator_id'].astype('string')
data["intiator"] = data["employee_id"] == data["initiator_id"]

print("====== Preparing the dataFrame finished =====")

filnal_data = pd.pivot_table(data, values='call', index=['employee_id'],
                 columns=['intiator'], aggfunc=np.mean).reset_index()

print("====== Pivoting finished =====")

with open('mapreduce_abramov.tsv', 'w') as f:
    for i in filnal_data.values.tolist()[:10]:
        f.write("{0}\t{1}\t{2}".format(i[0], int(i[1]), int(i[2])))
        f.write('\n')
        print("{0}\t{1}\t{2}".format(i[0], int(i[1]), int(i[2])))

====== Reading finished =====
====== Countind work duration finished =====
====== Preparing the dataFrame finished =====
====== Pivoting finished =====
100089558362	1898	1086
10076637711	1738	2158
100920206415	1751	2630
101997056684	1982	1757
102152760909	1915	2769
102304861721	1958	1823
102558754326	1748	1634
10258992785	1935	1791
102869392076	2281	2672
103045830455	1912	1892


In [5]:
filnal_data['False'] = filnal_data['False'].astype("integer")

KeyError: 'False'

In [26]:
filnal_data

intiator,employee_id,False,True
0,100089558362,1898.272727,1086.000000
1,10076637711,1738.682540,2158.428571
2,100920206415,1751.133333,2630.333333
3,101997056684,1982.696970,1757.333333
4,102152760909,1915.852941,2769.200000
...,...,...,...
2022,996118331930,1974.090909,1561.500000
2023,998669570651,1835.898876,1745.666667
2024,999029788377,1748.888889,1702.636364
2025,999438333915,1979.447059,2156.894737


In [25]:
filnal_data.columns

Index(['employee_id', False, True], dtype='object', name='intiator')

In [43]:
lst = []
with open('compare_result.txt', 'r') as f:
    for line in f.readlines():
        _l = line.split('\t')
        lst.append([_l[0], int(_l[1]), int(_l[2].strip())])

In [44]:
lst

[['100089558362', 1898, 1086],
 ['10076637711', 1738, 2158],
 ['100920206415', 1751, 2630],
 ['101997056684', 1982, 1757],
 ['102152760909', 1915, 2769],
 ['102304861721', 1958, 1823],
 ['102558754326', 1748, 1634],
 ['10258992785', 1935, 1791],
 ['102869392076', 2281, 2672],
 ['103045830455', 1912, 1892]]